# ROOT

Display this notebook in slidshow form by running
```
jupyter nbconvert ROOT.ipynb --to slides --post serve
```

### Starting up pyROOT

Install ROOT from [https://root.cern/install/](https://root.cern/install/)

In [2]:
import ROOT

Welcome to JupyROOT 6.28/04


## Data Processing

* ROOT is a database framework used by HEP
* Optimized for storing and processing independent *events*
* C++ library with python interface

p.s. when processing data, you should use C++ if you know it. Python is way slower, though is the better choice when plotting. We'll use python for this tutorial for simplicity. 

### Open a test file

This file is included in the git repo

In [6]:
in_file = ROOT.TFile('test.root')
in_file.ls()

TFile**		test.root	
 TFile*		test.root	
  KEY: TTree	tree;1	tree


Contains a single `TTree` called `tree`

### TTrees

* The main data storage class in ROOT is the TTree
* Can think of as a table
* Each column (TBranch) stores some data type
* Each row (entry) represents one event

| Entry | Animal | Legs |
| :-- | :-- | :-- |
| 0 | Whale | 0 |
| 1 | Human | 2 |
| 2 | Dog | 4 |

In [8]:
tree = in_file.Get('tree')
tree.GetEntries()

156

### TTrees (2)

List contents of tree:

In [ ]:
tree.Print()

```
******************************************************************************
*Tree    :tree      : tree                                                   *
*Entries :      156 : Total =           30015 bytes  File  Size =      17696 *
*        :          : Tree compression factor =   1.08                       *
******************************************************************************
*Br    0 :sample    : string                                                 *
*Entries :      156 : Total  Size=       1666 bytes  File Size  =        392 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=   2.99     *
*............................................................................*
*Br    1 :weight    : weight/F                                               *
*Entries :      156 : Total  Size=       1186 bytes  File Size  =        690 *
*Baskets :        1 : Basket Size=      32000 bytes  Compression=   1.01     *
*............................................................................*
```

### TTrees (3)

Loop over tree contents:

In [18]:
for entry,event in enumerate(tree):
    if entry >= 5: break
    print(entry, event.sample, event.weight)

0 WW 0.15749430656433105
1 WW 0.1984470635652542
2 WW 0.14358942210674286
3 WW 0.22988080978393555
4 WW 0.23132875561714172


### Exercise 1

* Note the branch `truth_lep1_pdgId`
* This is the pdgId of a lepton in each event (11 = e, 13 = mu, 15 = tau)
* Count the number of occurences of each lepton

In [29]:
# fill me!

* Answer: 74 electron, 75 muon, 7 tau

### Exercise 2

* Write a function that returns the mass of the `lep1`. Take `event` as an argument.
* Use the function to print the entry number and the mass for the 7 tau events.
* Hint 1: Create a [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html) with `ROOT.TLorentzVector(px, py, pz, e)`
* Hint 2: Search the documentation for how to retrieve the mass

In [30]:
def truth_lep1_mass(event):
    '''
    Gets the mass of 'truth_lep1' from an event in our test file.
    '''
    pass # TODO

# implement me!

Optional advanced exercise: generalize the function to any of the objects in the file

Answer: 
```
10 1.777402522020421
50 1.7730776138898674
56 1.777790404945941
105 1.7773254352322867
116 1.777033892011321
127 1.7769968885935143
132 1.7777922133134614
```

Advanced exercise solution:

In [ ]:
def mass(event, object):
    '''
    Gets the mass of an object from an event in our test file.
    @param object 
        The object name prefix, like 'truth_lep1'.
    '''
    par_names = ['px', 'py', 'pz', 'e']
    pars = [getattr(event, f'{object}_{x}') for x in par_names]
    v = ROOT.TLorentzVector(*pars)
    return v.M()